In [1]:
import sys

! "{sys.executable}" -m pip install azure-storage


In [2]:
# https://docs.microsoft.com/en-us/python/api/azure.storage.file.fileservice.fileservice?view=azure-python 
import azure.storage.file.models
from azure.storage.file import FileService
import os


#Azure File Storage settings
#update values of blobname, blobkey
blobname="Azure File Storage blob name"
blobkey="Azure File Storage key"
sharefolder='sharefoldername'

In [3]:
file_service = FileService(account_name=blobname, account_key=blobkey) 

In [4]:
def Upload(local_path, storage_sharefolder, storage_directoryname, overwrite=False):
    file_service.create_share(storage_sharefolder)
    if os.path.isdir(local_path):
        file_service.create_directory(storage_sharefolder, storage_directoryname)
        for root, dirs, files in os.walk(local_path):
            for file in files:
                try:
                    uploadfile(os.path.join(root,file),  storage_sharefolder, storage_directoryname, file,overwrite)
                except Exception as inst:
                    print("error, ",inst)        
                    continue
    else:
        filepath, file = os.path.split(local_path)
        uploadfile(local_path,  storage_sharefolder, storage_directoryname, file,overwrite)

In [5]:
def uploadfile(local_filepath,  storage_sharefolder, storage_directoryname, storage_filename, overwrite):
    if( overwrite or not file_service.exists(share_name= storage_sharefolder, directory_name= storage_directoryname, file_name=storage_filename)):
        print("uploading %s to %s/%s/%s" %(local_filepath,storage_sharefolder,storage_directoryname,storage_filename))
        file_service.create_file_from_path(share_name=storage_sharefolder,
                                           directory_name=storage_directoryname, 
                                           file_name=storage_filename, 
                                           local_file_path=local_filepath)

In [6]:
def downloadfile(local_filepath, storage_sharefolder, storage_directoryname, storage_filename, overwrite):
    if( overwrite or not os.path.exists(local_filepath) ):
        print("downloading %s/%s/%s to %s" %(storage_sharefolder,storage_directoryname,storage_filename,local_filepath))
        file_service.get_file_to_path(storage_sharefolder, storage_directoryname, file_name= storage_filename, file_path=local_filepath, open_mode='wb')


In [7]:
def download(local_path, storage_sharefolder, storage_directoryname, storage_filename=None, overwrite=False):
    
    if( not os.path.exists(local_path) ):
        os.makedirs(local_path)
    
    if storage_filename == None:
        generator = file_service.list_directories_and_files(storage_sharefolder,storage_directoryname)
        for file_or_dir in generator:
            try:
                file_path=os.path.join(local_path,file_or_dir.name)
                downloadfile(file_path, storage_sharefolder,storage_directoryname,file_or_dir.name,overwrite)
            except Exception as inst:
                print("error, ",inst)        
                continue
    else:
        file_path=os.path.join(local_path,storage_filename)
        downloadfile(file_path, storage_sharefolder, storage_directoryname, storage_filename, overwrite)   
        


## how it works

You can upload a file or folder to azure files

In [8]:
#Upload a single file
Upload('./logs/train1/dummytext1.txt',sharefolder,'train1',overwrite=True)
#upload a folder
Upload('./logs/train1',sharefolder,'train1',overwrite=True)


uploading ./logs/train1/dummytext1.txt to sharefoldername/train1/dummytext1.txt
uploading ./logs/train1\dummytext1.txt to sharefoldername/train1/dummytext1.txt
uploading ./logs/train1\dummytext2.txt to sharefoldername/train1/dummytext2.txt
uploading ./logs/train1\dummytext3.txt to sharefoldername/train1/dummytext3.txt


You can download a file or a folder from azure files

In [10]:
download('./downloaded_logs/t1',sharefolder,'train1',storage_filename='dummytext2.txt',overwrite=True)
#It download files/dirs under provided folder from azure storage/sharefolder to provided local path
download('./downloaded_logs/training1',sharefolder,'train1',storage_filename=None,overwrite=True)

downloading sharefoldername/train1/dummytext2.txt to ./downloaded_logs/t1\dummytext2.txt
downloading sharefoldername/train1/dummytext1.txt to ./downloaded_logs/training1\dummytext1.txt
downloading sharefoldername/train1/dummytext2.txt to ./downloaded_logs/training1\dummytext2.txt
downloading sharefoldername/train1/dummytext3.txt to ./downloaded_logs/training1\dummytext3.txt
